# Load all the .npy files for each gesture into a single dataset

In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences

gesture_labels = {"fist": 0, "swipe_left": 1, "wave": 2} #Đổi thành các nhãn bạn đã quay <3

base_path = "./gesture_videos/" #Đổi thành đường dẫn ở máy bạn <3
X = []  
y = []  

for gesture in gesture_labels.keys():
    gesture_path = os.path.join(base_path, gesture, "landmarks")
    if os.path.exists(gesture_path):
        for file in os.listdir(gesture_path):
            if file.endswith("_landmarks.npy"):
                gesture_name = gesture
                
                # Load the landmark data
                data = np.load(os.path.join(gesture_path, file))
                X.append(data)  
                y.append(gesture_labels[gesture_name]) 

max_length = 150 
X = pad_sequences(X, maxlen=max_length, padding='post', dtype='float32', value=0.0)

X = np.array(X) 
y = np.array(y)

np.save("../gesture_videos/X.npy", X) 
np.save("../gesture_videos/y.npy", y)  

print("X shape:", X.shape) 
print("y shape:", y.shape)  

# Step 5: Train a Machine Learning Model for Gesture Detection

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.load("../gesture_videos/X.npy") 
y = np.load("../gesture_videos/y.npy")  

num_classes = len(gesture_labels) 

y_encoded = to_categorical(y, num_classes)

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=37)

print("X_train shape:", X_train.shape)  
print("y_train shape:", y_train.shape) 
print("X_val shape:", X_val.shape)  
print("y_val shape:", y_val.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

num_classes = len(gesture_labels) 

model = Sequential([
    LSTM(64, input_shape=(150, 63), return_sequences=True),  
    Dropout(0.2),  
    LSTM(32), 
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax') 
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=20,  
                    batch_size=32, 
                    validation_data=(X_val, y_val),  
                    verbose=1)  

#Lưu em AI lại để khiển
model.save('../gesture_videos/model.h5')

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()